<img src="./logo.png" alt="Header" align="right" style="width: 100px;"/>

<center>
    <font size="+2">
    <b>AI Project 2<br></b>
        <b>Connect-4</b>
    </font>
</center>



In [1]:
# Wolf Assi ; Raoul Boulos ; Joe Mechref

# Introduction

From Wikipedia: Connect Four (also known as Four Up, Plot Four, Find Four, Four in a Row, Four in a Line, Drop Four, and Gravitrips in the Soviet Union) is a two-player connection board game, in which the players choose a color and then take turns dropping colored discs into a seven-column, six-row vertically suspended grid. The pieces fall straight down, occupying the lowest available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs. Connect Four is a solved game. The first player can always win by playing the right moves.

This is a python GUI where you can play Connect-4 1 on 1. However, what we want to do is to implement a smart AI agent to play against us. 

Run the below cell to import the libraries that we need.

In [1]:
import numpy as np
import pygame
import sys
import math

BLUE = (0,0,255)
BLACK = (0,0,0)
RED = (255,0,0)
YELLOW = (255,255,0)

ROW_COUNT = 6
COLUMN_COUNT = 7


pygame 2.0.0 (SDL 2.0.12, python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Helper Function

Below are some helper functions that you will use during your implementation, so read them and understand what they do but do not change them. 

In [2]:
def create_board(): #Don't change
    board = np.zeros((ROW_COUNT,COLUMN_COUNT))
    return board

# places a piece on the board
def drop_piece(board, row, col, piece): 
    board[row][col] = piece

# check if your move is valid
def is_valid_location(board, col):
    return board[ROW_COUNT-1][col] == 0

# utility function to be used in getLegalActions
def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == 0:
            return r

# returns a list of tuples representing the legal actions
# [(0,0), (0,1)] means that the only actions available are
# row 0, column 0 and row 0, column 1
# you only need the column number for the action. 
def getLegalActions(board):
        actions=[]
        for col in range(7):
                row=get_next_open_row(board, col)
                
                if row is not None:
                        actions.append((row,col))
        return actions

def print_board(board):
    print(np.flip(board, 0))

# function that returns true if the board is a winning position
# for the player designated by piece
# piece=1 is human player, piece=2 is AI. 
def winning_move(board, piece):
    # Check horizontal locations for win
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True

    # Check vertical locations for win
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT-3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True

    # Check positively sloped diaganols
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT-3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True

    # Check negatively sloped diaganols
    for c in range(COLUMN_COUNT-3):
        for r in range(3, ROW_COUNT):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True
            
# For visualization only
def draw_board(board):
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, BLUE, (c*SQUARESIZE, r*SQUARESIZE+SQUARESIZE, SQUARESIZE, SQUARESIZE))
            pygame.draw.circle(screen, BLACK, (int(c*SQUARESIZE+SQUARESIZE/2), int(r*SQUARESIZE+SQUARESIZE+SQUARESIZE/2)), RADIUS)
    
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):        
            if board[r][c] == 1:
                pygame.draw.circle(screen, RED, (int(c*SQUARESIZE+SQUARESIZE/2), height-int(r*SQUARESIZE+SQUARESIZE/2)), RADIUS)
            elif board[r][c] == 2: 
                pygame.draw.circle(screen, YELLOW, (int(c*SQUARESIZE+SQUARESIZE/2), height-int(r*SQUARESIZE+SQUARESIZE/2)), RADIUS)
    pygame.display.update()


# Eval Function

As you will notice later, running minimax at a depth of more than 5 or 6 will result in a very slow game. Hence it wont be possible to build the whole tree from the begining. This is why you will implement a depth limited minimax and you will need an evaluation function. You could leave this part till the end if you want but you will need to implement it, otherwise your agent will not be able to find its way to victory. 

A good evaluation function is given by just counting the number of possible 4 in a rows that each player can still make and substract that from each other. You might come up with a different evaluation function. The important thing is that you should never be able to beat it.

In [3]:
def Eval(board):
        total=0
        player = 1 
        piece = 2
        Temp=board.tolist()
        print(type(Temp))
        # Implement a good evaluation function here
            # Check horizontal locations
        for c in range(COLUMN_COUNT-3):
            for r in range(ROW_COUNT):
                if Temp[r][c] == piece and Temp[r][c+1] == piece and Temp[r][c+2] == piece and Temp[r][c+3] == piece:
                    total += 100
                if Temp[r][c] == piece and Temp[r][c+1] == piece and Temp[r][c+2] == piece:
                    total += 10
                if Temp[r][c] == piece and Temp[r][c+1] == piece:
                    total += 5
                if Temp[r][c] == player and Temp[r][c+1] == player and Temp[r][c+2] == player and Temp[r][c+3] == player:
                    total -= 100
                if Temp[r][c] == player and Temp[r][c+1] == player and Temp[r][c+2] == player:
                    total -= 10
                if Temp[r][c] == player and Temp[r][c+1] == player:
                    total -= 5
                

        # Check vertical locations
        for c in range(COLUMN_COUNT):
            for r in range(ROW_COUNT-3):
                if Temp[r][c] == piece and Temp[r+1][c] == piece and Temp[r+2][c] == piece and Temp[r+3][c] == piece:
                    total += 100
                if Temp[r][c] == piece and Temp[r+1][c] == piece and Temp[r+2][c] == piece:
                    total += 10
                if Temp[r][c] == piece and Temp[r+1][c] == piece:
                    total += 5
                if Temp[r][c] == player and Temp[r+1][c] == player and Temp[r+2][c] == player and Temp[r+3][c] == player:
                    total -= 100
                if Temp[r][c] == player and Temp[r+1][c] == player and Temp[r][c] == player:
                    total -= 10
                if Temp[r][c] == player and Temp[r+1][c] == player:
                    total -= 5

        # Check positively sloped diaganols
            for c in range(COLUMN_COUNT-3):
                for r in range(ROW_COUNT-3):
                    if Temp[r][c] == piece and Temp[r+1][c+1] == piece and Temp[r+2][c+2] == piece and Temp[r+3][c+3] == piece:
                        total += 100
                    if Temp[r][c] == piece and Temp[r+1][c+1] == piece and Temp[r+2][c+2] == piece:
                        total += 10
                    if Temp[r][c] == piece and Temp[r+1][c+1] == piece:
                        total += 5
                    if Temp[r][c] == player and Temp[r+1][c+1] == player and Temp[r+2][c+2] == player and Temp[r+3][c+3] == player:
                        total -= 100
                    if Temp[r][c] == player and Temp[r+1][c+1] == player and Temp[r+2][c+2] == player:
                        total -= 10
                    if Temp[r][c] == player and Temp[r+1][c+1] == player:
                        total -= 5

        # Check negatively sloped diaganols
        for c in range(COLUMN_COUNT-3):
            for r in range(3, ROW_COUNT):
                if Temp[r][c] == piece and Temp[r-1][c+1] == piece and Temp[r-2][c+2] == piece and Temp[r-3][c+3] == piece:
                    total += 100
                if Temp[r][c] == piece and Temp[r-1][c+1] == piece and Temp[r-2][c+2] == piece:
                    total += 10
                if Temp[r][c] == piece and Temp[r-1][c+1] == piece:
                    total += 5
                if Temp[r][c] == player and Temp[r-1][c+1] == player and Temp[r-2][c+2] == player and Temp[r-3][c+3] == player:
                    total -= 100
                if Temp[r][c] == player and Temp[r-1][c+1] == player and Temp[r-2][c+2] == player:
                    total -= 10
                if Temp[r][c] == player and Temp[r-1][c+1] == player:
                    total -= 5
                
        return total

# Minimax

Implement a depth limited $\alpha-\beta$ minimax

In [4]:
def minimax(board,player,alfa,beta,depth):
    # implement the alfa beta pruning version of minimax
    # you should take care that this is a depth limited search
    # this function must return the value of the board parameter
    # player=1 is the human player and player=2 is AI.
    available_actions = getLegalActions(board)
    if depth == 0:
        return Eval(board)
    
    if winning_move(board,2):
            return 100
        
    if winning_move(board,1):
            return -100
        
    if player == 2:
        value= -math.inf
        for action in available_actions:
            print("Player 2's turn")
            tmp = board.copy()
            drop_piece(tmp,action[0],action[1],2)
            value = max(value,minimax(tmp,1,alfa,beta,depth-1))
            if value > beta:
                return value
            alfa = max(alfa,value)
        return value
                

    if player == 1:
        value = math.inf
        for action in available_actions:
            print("Player 1's turn")
            tmp = board.copy()
            drop_piece(tmp,action[0],action[1],1)
            value = min(value,minimax(tmp,2,alfa,beta,depth-1))
            if value < alfa:
                return value
            beta = min(beta,value)
        return value


# Get Best Action

Remember you need to implement this function for your minimax to be useful.

In [5]:
import random
def GetBestAction(board,depth):
    # this will return a random action
    # Change this function to return the best minimax action
    # at the designated depth
    actions=getLegalActions(board)
    score=-math.inf
    for action in actions:
        tmp = board.copy()
        drop_piece(tmp,action[0],action[1],2)
        value = minimax(tmp,1,-math.inf,math.inf,depth-1)
        #print(value)
        #print(tmp)
        if (value>score):
            score = value
            bestAction = action[1]
    return bestAction , score
#   print(actions)
#    score=0
#    idx=random.randint(0,len(actions)-1)
#    bestAction=actions[idx][1]
#    return bestAction,score

# Run

Run the cell below to test your implementation. You can run this cell without changing anything to play against a random player (getBestAction) is random. 

In [6]:
board = create_board()
print_board(board)
print(Eval(board))
game_over = False
turn = 0

pygame.init()

SQUARESIZE = 100

width = COLUMN_COUNT * SQUARESIZE
height = (ROW_COUNT+1) * SQUARESIZE

size = (width, height)

RADIUS = int(SQUARESIZE/2 - 5)

screen = pygame.display.set_mode(size)
draw_board(board)
pygame.display.update()

myfont = pygame.font.SysFont("monospace", 75)

while not game_over:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()

        if event.type == pygame.MOUSEMOTION:
            pygame.draw.rect(screen, BLACK, (0,0, width, SQUARESIZE))
            posx = event.pos[0]
            if turn == 0:
                pygame.draw.circle(screen, RED, (posx, int(SQUARESIZE/2)), RADIUS)
            
        pygame.display.update()

        if event.type == pygame.MOUSEBUTTONDOWN:
            pygame.draw.rect(screen, BLACK, (0,0, width, SQUARESIZE))
            #print(event.pos)
            # Ask for Player 1 Input
            if turn == 0:
                posx = event.pos[0]
                col = int(math.floor(posx/SQUARESIZE))

                if is_valid_location(board, col):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, 1)

                    if winning_move(board, 1):
                        label = myfont.render("Player 1 wins!!", 1, RED)
                        screen.blit(label, (40,10))
                        game_over = True
                #print_board(board)
                draw_board(board)

                turn += 1
                turn = turn % 2


    # # AI Turn
    if turn==1 and not game_over:

        col,score = GetBestAction(board,5)

        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, 2)

            if winning_move(board, 2):
                label = myfont.render("Player 2 wins!!", 1, YELLOW)
                screen.blit(label, (40,10))
                game_over = True

        
        draw_board(board)
        turn += 1
        turn = turn % 2

    if game_over:
        pygame.time.wait(3000)
        pygame.display.quit()
        pygame.quit()


[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
<class 'list'>
0
Player 1's turn
Player 2's turn
Player 1's turn
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 1's turn
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 1's turn
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 2's turn
<class 'list'>
Player 1's turn
Player 2's turn
<class 'list'>
Player 1's turn
Player 2's turn
<class 'list'>
Player 1's turn
Player 2's turn
<class 'list'>
Player 1's turn
Player 2's turn
<class 'list'>
Player 2's turn
Player 1's turn
Player 2's turn
<class 'list'>
Player 2's 

SystemExit: 

D:\AIFILE\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
